<a href="https://colab.research.google.com/github/dakyommii/study/blob/main/dduck/KeyBERT_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Korean KeyBert

키버트 실습을 위해서는 우선 SBERT를 위한 패키지인 sentence_transformers와 형태소 분석기 KoNLPy를 설치합니다.

In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 4.2 MB 10.6 MB/s 
     |████████████████████████████████| 1.2 MB 50.9 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 70.7 MB/s 
     |████████████████████████████████| 6.6 MB 50.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=e8aaa5594bd79d4ab11bd0cea85c48306ef36193b8d7904af875112cf88e7529
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 69.0 MB/s 


###1. 기본 KeyBERT

In [3]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [4]:
doc = """
저는 성격상 낯을 가리는 경향이 있기 때문에 모르는 사람을 많이 만나서 응대하는 것 보다 백엔드 개발을 선호합니다. 또한 저는 백엔드 개발을 하고 싶어서 해당 직무에 지원한 만큼 백엔드 개발을 선호합니다.  
"""

형태소 분석기를 통해 명사만 추출한 문서를 만듭니다.

In [5]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('\n', 'Foreign'), ('저', 'Noun'), ('는', 'Josa'), ('성격', 'Noun'), ('상', 'Suffix'), ('낯', 'Noun'), ('을', 'Josa'), ('가리는', 'Verb'), ('경향', 'Noun'), ('이', 'Josa')]
명사 추출 : 저 성격 낯 경향 때문 사람 응대 것 백엔드 개발 선호 또한 저 백엔드 개발 해당 직무 지원 만큼 백엔드 개발 선호


여기서는 사이킷런의 CountVectorizer를 사용하여 단어를 추출합니다. CountVectorizer를 사용하는 이유는 n_gram_range의 인자를 사용하면 쉽게 n-gram을 추출할 수 있기 때문입니다. 예를 들어, (2, 3)으로 설정하면 결과 후보는 2개의 단어를 한 묶음으로 간주하는 bigram과 3개의 단어를 한 묶음으로 간주하는 trigram을 추출합니다.

In [6]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 29
trigram 다섯개만 출력 : ['개발 선호' '개발 선호 또한' '개발 해당' '개발 해당 직무' '경향 때문']


문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화 -> 한국어를 포함하고 있는 다국어 SBERT 로드

In [7]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

문서와 가장 유사한 키워드들을 추출합니다. 여기서는 문서와 가장 유사한 키워드들은 문서를 대표하기 위한 좋은 키워드라고 가정합니다. 상위 5개의 키워드를 출력합니다

In [8]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['만큼 백엔드 개발', '성격 경향 때문', '백엔드 개발 선호', '응대 백엔드 개발', '선호 또한 백엔드']


5개의 키워드가 출력되는데, 이들의 의미가 좀 비슷해보입니다. 비슷한 의미의 키워드들이 리턴되는 이유는 이 키워드들이 문서를 가장 잘 나타내고 있기 때문입니다. 만약, 지금 출력한 것보다는 좀 더 다양한 의미의 키워드들이 출력된다면 이들을 그룹으로 본다는 관점에서는 해당 키워드들이 문서를 잘 나타낼 가능성이 적을 수도 있습니다. 따라서 키워드들을 다양하게 출력하고 싶다면 키워드 선정의 정확성과 키워드들의 다양성 사이의 미묘한 균형이 필요합니다.

###2. Max Sum Similarity

데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의
의도: 후보 간의 유사성 최소화하면서 문서와의 후보 유사성 극대화

In [9]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

이를 위해 상위 10개의 키워드를 선택하고 이 10개 중에서 서로 가장 유사성이 낮은 5개를 선택합니다. 낮은 nr_candidates를 설정하면 결과는 출력된 키워드 5개는 기존의 코사인 유사도만 사용한 것과 매우 유사한 것으로 보입니다.

In [10]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['경향 때문 사람', '또한 백엔드 개발', '성격 경향', '사람 응대 백엔드', '백엔드 개발 선호']

그러나 상대적으로 높은 nr_candidates는 더 다양한 키워드 5개를 만듭니다.

In [11]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['경향 때문', '백엔드 개발', '직무 지원', '또한 백엔드 개발', '성격 경향']

###3. Maximal Marginal Relevance

MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력
1. 문서와 가장 유사한 키워드/키프레이즈를 선택
2. 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보를 반복적으로 선택

In [12]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

만약 상대적으로 낮은 diversity 값을 설정한다면, 결과는 기존의 코사인 유사도만 사용한 것과 매우 유사

In [13]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['선호 또한 백엔드', '응대 백엔드 개발', '성격 경향 때문', '백엔드 개발 선호', '만큼 백엔드 개발']

In [14]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['선호 또한 백엔드', '개발 선호', '성격 경향', '백엔드 개발', '직무 지원']